In [8]:
#IMPLEMENTACIÓN DEL MÉTODO DE CONJUNTOS ACTIVOS EN LA SOLUCIÓN DEL PROBLEMA(P)

import numpy as np

# Definimos la matriz H, asociada al problema (P)
H = np.array([
    [12983, 0, 0, 0],
    [0, 2400, 0, 0],
    [0, 0, 440, -53],
    [0, 0, -53, 68.7]
], dtype=float)

# Se define el vector del término lineal del problema (P)
f = np.array([-6748, -1184, -420, -70], dtype=float)

# Matriz de restricciones Gx <= h
G = np.array([
    [-259.64, -960, -70.25, 0.513],
    [-584.2, -24, -55.2, -5.94],
    [4.82, 0.32, 0.21, 0.07],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 220, -26],
    [0, 0, -27, 34]
], dtype=float)

# Lado derecho de las restricciones
h = np.array([-781.62, -35, 1.939, 1.039, 0.987, 420, 70], dtype=float)


##############################
#Implementación del Algoritmo 14.3 del texto usado en clase (Conjuntos Activos)
#########################



def conjuntos_activos(H, f, G, h, p0, tol=1e-3, max_iter=50):

    n = len(p0)   # número de variables
    m = len(h)    # número de restricciones

    # Se inicia con x0=p0 factible y A0
    x = p0.copy()
    A = []        # conjunto activo A_k (índices)
    
    for k in range(max_iter):

        # Evaluamos las restricciones: g_i(x) = a_i^T x - h_i
        g = G @ x - h

        
        # Se resuelve el QP reducido (tripleta KKT)
        
        if len(A) > 0:
            A_mat = G[A]   #restricciones activas

            # Sistema KKT reducido
            KKT = np.block([
                [H, A_mat.T],
                [A_mat, np.zeros((len(A), len(A)))]
            ])

            # Lado derecho del sistema
            rhs = np.concatenate([-(H @ x + f),-g[A]])

            sol = np.linalg.solve(KKT, rhs) #resuelve el sistema KKT

            d = sol[:n]     # dirección de búsqueda d^k
            mu = sol[n:]    # multiplicadores asociados a A_k

        else:
            
            #Se considera el caso sin restricciones activas
            d = -np.linalg.solve(H, H @ x + f)
            mu = np.array([])

        
        # Si d^k = 0, verificar KKT
        ##########################
        if np.linalg.norm(d) < tol:

            # Si no hay restricciones activas, ya es óptimo
            if len(A) == 0:
                return x, np.zeros(m), A, k+1

            # Si todos los multiplicadores son no negativos entonces se tiene el óptimo
            if np.all(mu >= -tol):
                lambda_1 = np.zeros(m)
                for i, idx in enumerate(A):
                    lambda_1[idx] = mu[i]
                return x, lambda_1, A, k+1

            # Si hay multiplicadores negativos entonces se eliminar la restricción
            j = np.argmin(mu)
            del A[j]
            continue

        ########
        # Cálculo de la longitud de paso máximo factible
        #########
        alpha = 1.0    #tamaño de paso inicial
        j_inicial = None  #restriccion factible

        for i in range(m):
            if i not in A:   #se considera las condiciones inactivas
                ai = G[i]
                denom = ai @ d  #caculo de a^T d^k

                if denom > tol:
                    alpha_i = (h[i] - ai @ x) / denom  #paso máximo antes de violar la restriccion
                    if alpha_i < alpha:
                        alpha = alpha_i      #guardamos el paso minimo factible
                        j_inicial = i             

        # Actualizar punto
        x = x + alpha * d

        # Si una restricción se activa exactamente
        if j_inicial is not None:
            A.append(j_inicial)

    raise RuntimeError("El algoritmo no convergió")


# costos iniciales
p0 = np.array([0.297, 0.720, 1.050, 0.815])

# Ejecutar algoritmo
optimo, lambdas, A_final,iteraciones = conjuntos_activos(H, f, G, h, p0)

# Resultados
print("Solución óptima p*:")
print(optimo*1000)

print("\nMultiplicadores de Lagrange λ:")
print(lambdas)

print("\nConjunto activo final (índices):")
print(A_final)

print("\nNúmero de iteraciones:")
print(iteraciones)

print("\nValor óptimo:")
print(-1000*(0.5 * optimo @ H @ optimo + f @ optimo))


Solución óptima p*:
[ 303.53193     667.4149858   891.71913059 1073.46120755]

Multiplicadores de Lagrange λ:
[  0.64084972   0.         616.93675797   0.           0.
   0.           0.        ]

Conjunto activo final (índices):
[2, 0]

Número de iteraciones:
4

Valor óptimo:
1991728.2719804228
